In [61]:
from sklearn import datasets, linear_model, metrics
from sklearn.model_selection import cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn import model_selection

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

### Генерация датасета

In [62]:
iris = datasets.load_iris()

In [99]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(iris.data, iris.target, 
                                                                                     test_size = 0.3,random_state = 0)

### Выбор модели

In [101]:
classifier = linear_model.SGDClassifier(random_state = 0)

### Генерация сетки

In [103]:
classifier.get_params().keys()  #посмотрим какие параметры мы можем настраивать

dict_keys(['alpha', 'average', 'class_weight', 'early_stopping', 'epsilon', 'eta0', 'fit_intercept', 'l1_ratio', 'learning_rate', 'loss', 'max_iter', 'n_iter_no_change', 'n_jobs', 'penalty', 'power_t', 'random_state', 'shuffle', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

In [110]:
parameters_grid = {
    'loss' : ['hinge', 'log', 'squared_hinge', 'squared_loss'],
    'penalty' : ['l1', 'l2'],
    'max_iter' : range(5,10),
    'alpha' : np.linspace(0.0001, 0.001, num = 5),
}

In [111]:
cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 0) # cv это выбранная нами стратегия кроссвалидации

### Подбор параметров и оценка качества

In [112]:
grid_cv = GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv=cv)

In [113]:
%%time
grid_cv.fit(train_data, train_labels)

CPU times: total: 11.9 s
Wall time: 12 s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
             estimator=SGDClassifier(random_state=0),
             param_grid={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.001   ]),
                         'loss': ['hinge', 'log', 'squared_hinge',
                                  'squared_loss'],
                         'max_iter': range(5, 10), 'penalty': ['l1', 'l2']},
             scoring='accuracy')

In [114]:
print (grid_cv.best_score_)
print (grid_cv.best_params_)

0.9047619047619048
{'alpha': 0.0007750000000000001, 'loss': 'hinge', 'max_iter': 9, 'penalty': 'l1'}


### Randomized grid search

In [115]:
randomized_grid_cv = RandomizedSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv, n_iter = 20, 
                                                   random_state = 0)

In [116]:
randomized_grid_cv.fit(train_data, train_labels)

RandomizedSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
                   estimator=SGDClassifier(random_state=0), n_iter=20,
                   param_distributions={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.001   ]),
                                        'loss': ['hinge', 'log',
                                                 'squared_hinge',
                                                 'squared_loss'],
                                        'max_iter': range(5, 10),
                                        'penalty': ['l1', 'l2']},
                   random_state=0, scoring='accuracy')

In [117]:
print (randomized_grid_cv.best_score_)
print (randomized_grid_cv.best_params_)

0.8666666666666666
{'penalty': 'l1', 'max_iter': 8, 'loss': 'squared_hinge', 'alpha': 0.0007750000000000001}
